In [10]:
options(repr.plot.width = 20)
options(repr.plot.height = 6)

In [11]:
library(zoo)
library(ggplot2)
library(dplyr)
library(forecast)
library(dynlm)
library(urca)
library(fUnitRoots)
library(ggfortify)
library(data.table)
library(tseries)
library(dplyr)
library(aTSA)
library(lmtest)
library(readxl)
library(vars)
library(tsDyn)
library(MTS)

Susikeliame duomenis:

In [14]:
data <- read.csv("Kursinis/duomenys.csv", header=T)

In [15]:
data$TIME <-as.Date(data$TIME, format("%Y-%m-%d"))

# STACIONARUMO TIKRINIMAS

Iš `data.frame`susikuriame laiko eilučių objektus.

In [16]:
unts<- ts(data$UN,start=c(1968,1),end=c(2018,4), frequency=4)

In [17]:
gdpts<- ts(data$GDP,start=c(1968,1),end=c(2018,4), frequency=4)

In [18]:
infts<- ts(data$INF,start=c(1968,1),end=c(2018,4), frequency=4)

## NEDARBO SKIRTUMAI

Vienetinės šaknies tikrinimas su funkcija `adf.test()`

In [19]:
adf.test(unts)

Augmented Dickey-Fuller Test 
alternative: stationary 
 
Type 1: no drift no trend 
     lag    ADF p.value
[1,]   0 -0.386   0.533
[2,]   1 -0.191   0.589
[3,]   2 -0.197   0.587
[4,]   3 -0.284   0.562
[5,]   4 -0.320   0.552
Type 2: with drift no trend 
     lag   ADF p.value
[1,]   0 -1.31   0.587
[2,]   1 -1.28   0.598
[3,]   2 -1.42   0.548
[4,]   3 -1.68   0.451
[5,]   4 -1.29   0.595
Type 3: with drift and trend 
     lag    ADF p.value
[1,]   0 -0.114    0.99
[2,]   1  0.851    0.99
[3,]   2  0.763    0.99
[4,]   3  0.207    0.99
[5,]   4  0.463    0.99
---- 
Note: in fact, p.value = 0.01 means p.value <= 0.01 


Kadangi p-reikšmės yra daugiau už 0.05 tai $ H_0 $ neatmetame. Eilutė turi vienetinę šaknį.

## BVP SKIRTUMAI

In [21]:
adf.test(gdpts)

Augmented Dickey-Fuller Test 
alternative: stationary 
 
Type 1: no drift no trend 
     lag   ADF p.value
[1,]   0 -8.67    0.01
[2,]   1 -5.40    0.01
[3,]   2 -4.14    0.01
[4,]   3 -4.45    0.01
[5,]   4 -3.70    0.01
Type 2: with drift no trend 
     lag    ADF p.value
[1,]   0 -10.75    0.01
[2,]   1  -6.78    0.01
[3,]   2  -5.22    0.01
[4,]   3  -5.56    0.01
[5,]   4  -4.74    0.01
Type 3: with drift and trend 
     lag    ADF p.value
[1,]   0 -12.71    0.01
[2,]   1  -8.21    0.01
[3,]   2  -6.44    0.01
[4,]   3  -6.74    0.01
[5,]   4  -6.04    0.01
---- 
Note: in fact, p.value = 0.01 means p.value <= 0.01 


BVP skirtumai vienetinės šaknies neturi.

Laiko eilutė stacionari. 

## Infliacija

In [23]:
adf.test(infts)

Augmented Dickey-Fuller Test 
alternative: stationary 
 
Type 1: no drift no trend 
     lag   ADF p.value
[1,]   0 -6.95  0.0100
[2,]   1 -2.79  0.0100
[3,]   2 -2.61  0.0100
[4,]   3 -1.69  0.0898
[5,]   4 -2.05  0.0416
Type 2: with drift no trend 
     lag   ADF p.value
[1,]   0 -8.19  0.0100
[2,]   1 -3.23  0.0210
[3,]   2 -3.04  0.0352
[4,]   3 -1.79  0.4102
[5,]   4 -2.35  0.1905
Type 3: with drift and trend 
     lag    ADF p.value
[1,]   0 -11.42  0.0100
[2,]   1  -4.65  0.0100
[3,]   2  -4.56  0.0100
[4,]   3  -2.62  0.3152
[5,]   4  -3.71  0.0247
---- 
Note: in fact, p.value = 0.01 means p.value <= 0.01 


dėl sezoniškumo rezultatas nelabai aiškus

In [26]:
sdiff <- diff(infts, lag = 4)

In [27]:
adf.test(sdiff)

Augmented Dickey-Fuller Test 
alternative: stationary 
 
Type 1: no drift no trend 
     lag   ADF p.value
[1,]   0 -9.73    0.01
[2,]   1 -6.64    0.01
[3,]   2 -5.30    0.01
[4,]   3 -9.31    0.01
[5,]   4 -7.93    0.01
Type 2: with drift no trend 
     lag   ADF p.value
[1,]   0 -9.71    0.01
[2,]   1 -6.63    0.01
[3,]   2 -5.30    0.01
[4,]   3 -9.30    0.01
[5,]   4 -7.94    0.01
Type 3: with drift and trend 
     lag   ADF p.value
[1,]   0 -9.69    0.01
[2,]   1 -6.61    0.01
[3,]   2 -5.28    0.01
[4,]   3 -9.28    0.01
[5,]   4 -7.92    0.01
---- 
Note: in fact, p.value = 0.01 means p.value <= 0.01 


In [28]:
dinfza <- ur.za(sdiff,model="trend")

In [29]:
summary(dinfza)


################################ 
# Zivot-Andrews Unit Root Test # 
################################ 


Call:
lm(formula = testmat)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.6072 -0.2126  0.0321  0.2219  4.0710 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.57551    0.28370   2.029   0.0439 *  
y.l1         0.33148    0.06681   4.961 1.52e-06 ***
trend       -0.02647    0.01208  -2.191   0.0296 *  
dt           0.02759    0.01261   2.187   0.0299 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.753 on 195 degrees of freedom
  (1 observation deleted due to missingness)
Multiple R-squared:  0.1481,	Adjusted R-squared:  0.135 
F-statistic:  11.3 on 3 and 195 DF,  p-value: 7.228e-07


Teststatistic: -10.006 
Critical values: 0.01= -4.93 0.05= -4.42 0.1= -4.11 

Potential break point at position: 27 


Infliacijos skirtumai yra staionarūs.